# User Segmentation RFM Model

- This exploration looks to use K-means clustering to cluster users based on an RFM (recency, frequency, monetary) model.
- It will clusters users based on the recency and frequency of objective completions, as well as the total number of points that they have claimed.
- Code for K-means clustering is from https://neptune.ai/blog/customer-segmentation-using-machine-learning.

#### Import packages

In [44]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
import numpy as np
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

#chart_studio credentials
chart_studio.tools.set_credentials_file(username = os.environ.get('cs_user'), api_key = os.environ.get('cs_key'))

In [45]:
#Pandas number of rows displayed (Set None for all rows)
#pd.set_option('display.max_rows', 10)

#### SQL Setup

In [46]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

## Identifying user features

We will be examining users sponsored by Red River Credit Union, as they have a wide selection of objectives to choose from.

In [47]:
rfm_query = '''
SELECT 
	id AS user_id, 
    DATEDIFF(current_date(), 
    MAX(date_claimed)) AS recency, 
    COUNT(name) AS frequency, 
    SUM(points) as points,
    IFNULL(num_modules, 0) AS num_modules,
    num_login
FROM
(
SELECT 
	u.id, 
    o.name,
    o.points, 
    oc.date_claimed
FROM objective_claim oc
LEFT JOIN user u ON u.id = oc.user_id
LEFT JOIN objective o ON o.id = oc.objective_id
WHERE u.institution_id = 19634
) objective_table
LEFT JOIN (
	SELECT ecm.user_id, COUNT(DISTINCT(ecm.education_module_id)) AS num_modules
	FROM education_completed_module ecm
	LEFT JOIN user u ON u.id = ecm.user_id
	WHERE u.institution_id = 19634
	GROUP BY ecm.user_id
) module_table ON module_table.user_id = objective_table.id
LEFT JOIN (
	SELECT lt.user_id, COUNT(DISTINCT(DATE(lt.date_created))) AS num_login
	FROM login_tracker lt
	LEFT JOIN user u ON u.id = lt.user_id
	WHERE u.institution_id = 19634
	GROUP BY lt.user_id
) login_table ON login_table.user_id = objective_table.id
GROUP BY id
'''

rfm_table = query(rfm_query)
rfm_table

,user_id,recency,frequency,points,num_modules,num_login
0,913442,258,4,600.0,6,2
1,916622,266,1,100.0,0,1
2,917246,250,15,2700.0,37,7
3,917379,258,3,600.0,6,4
4,934769,248,2,200.0,3,2
...,...,...,...,...,...,...
4150,2214010,0,2,300.0,4,1
4151,2214057,0,1,100.0,1,1
4152,2214146,0,1,100.0,3,1
4153,2214178,0,1,100.0,0,1


## Fitting the Model

In [48]:
#Define a K-means model:
kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)

#Train the model:
kmeans_model.fit(rfm_table[['recency','frequency','points']])

KMeans(max_iter=400, random_state=42)

### Finding the optimal number of clusters:

In [49]:
# Create the K means model for different values of K
def try_different_clusters(K, data):
       
    cluster_values = list(range(1, K+1))
    inertias=[]
    
    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)
    
    return inertias

In [50]:
# Find output for k values between 1 to 12 
outputs = try_different_clusters(12, rfm_table[['recency','frequency','points']])
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})


In [51]:
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),                  
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

As the plot above indicates, the optimal number of clusters is 4.

In [71]:
# Re-Train K means model with k=4
kmeans_model_new = KMeans(n_clusters = 4,init='k-means++',max_iter=400,random_state=42)
kmeans_model_new.fit_predict(rfm_table[['recency','frequency','points']])

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [72]:
# Create data arrays
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)

# Add "cluster" to customers data
points = np.append(points, [[0], [1], [2], [3]], axis=1)
rfm_table["clusters"] = kmeans_model_new.labels_
rfm_table

/Users/wgjin/Developer/Zogo/Notebooks/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning:

overflow encountered in expm1



,user_id,recency,frequency,points,num_modules,num_login,clusters
0,913442,258,4,600.0,6,2,0
1,916622,266,1,100.0,0,1,0
2,917246,250,15,2700.0,37,7,0
3,917379,258,3,600.0,6,4,0
4,934769,248,2,200.0,3,2,0
...,...,...,...,...,...,...,...
4150,2214010,0,2,300.0,4,1,0
4151,2214057,0,1,100.0,1,1,0
4152,2214146,0,1,100.0,3,1,0
4153,2214178,0,1,100.0,0,1,0


## Visualization the clusters

In [73]:
# visualize clusters
cluter_scatter = px.scatter_3d(rfm_table,
                    color='clusters',
                    x="recency",
                    y="frequency",
                    z="points",            
                    category_orders = {"clusters": ["0", "1", "2", "3"]}                    
                    )
cluter_scatter.update_layout()
cluter_scatter.show()

# Export to chart studio
py.plot(cluter_scatter, name = 'Segmentation of RRCU Users', auto_open = True)

'https://plotly.com/~woonggyujin/19/'

## How do the clusters differ in user behavior?
- Which clusters complete more education modules or log in more?
- How would we change the way we target each cluster?

In [74]:
rfm_table.groupby(['clusters'], as_index=False).median().drop('user_id', axis = 1)

,clusters,recency,frequency,points,num_modules,num_login
0,0,117.0,2.0,300.0,3.0,1.0
1,1,0.0,508.0,110225.0,962.5,172.0
2,2,0.0,109.0,21725.0,169.5,41.0
3,3,0.0,310.0,68200.0,543.0,112.0


Users in cluster 1 are our power users. As we might expect, their median number of modules, points and days logged in are far higher than the other clusters. The question is how we convert users from one cluster to the next, and how we are going to target each cluster differently.

### E-statements

Users who have completed e-statements:

In [77]:
estatement_query = '''
SELECT user_id
FROM objective_claim oc
WHERE oc.objective_id = 537
'''

estatement_user_table = query(estatement_query)

pd.merge(estatement_user_table, rfm_table, on='user_id', how='left')

,user_id,recency,frequency,points,num_modules,num_login,clusters
0,1097999,0,515,113200.0,932,174,1
1,1103592,0,553,123850.0,1000,180,1
2,1327069,0,375,91750.0,733,117,1
3,2174487,0,55,20050.0,116,17,2
4,2198662,0,41,15800.0,47,13,2
5,2198818,1,17,9400.0,25,5,0
6,2199981,0,20,10500.0,41,10,0
7,1211797,1,45,10400.0,3,40,0
8,2199334,5,7,6300.0,6,2,0
9,1294534,4,21,9900.0,56,7,0


None of the users that have completed E-statements are from user cluster 3. Users in cluster 3 are users that are sufficiently engaged to the app. We can deduce that these users are going to be willing to complete e-statements if we display the option to them aggressively.